<a href="https://colab.research.google.com/github/averkief/My_Date_Analytics/blob/main/Popular_Russian_syllables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Предисловие***<br>
Когда мой сын подрастал и начинал познавать мир, то дошло дело и до чтения.
Современные методики рекомендуют учить детей не буквам, а слогам. Указывая на то, что так будет проще научиться читать.<br>
И так в русском языке 10 гласных букв и 20 согласных. Отбросив соединения согласных с гласными которые не соответсвуют правилам русского языка, получаем 158 правильных слогов.<br>
И тут у меня возник вопрос, а какие слоги самые популярные, какие лучше выучить первыми???


**Последовательность действий**<br>
*Этап 1: Находжение источника*<br>
Выбор книги выпал на "Война и Мир" Л.Н.Толстова, как самая большая по объему. Текст книги есть в интернете. Требуется спарсить текст с сайта<br>
*Этап 2: Обработка сырых данных* <br>
Разбиваем текст на слова и извлекаем пунктуацию, предлоги, местоимения слова не относящиеся к словам со слогами<br>
*Этап 3: Внесение данных для нахождения слогов* <br>
Подготовка искомых слогов и счетчиков подсчета<br>
*Этап 4: Обработка полезных данных* <br>
Обрабатываем каждое слово<br>
*Этап 5: Вывод* <br>
Получение информации: количественное значиени, графики<br>

**С какими затруднениями пришлось столкнуться**<br>
**Загвозка 1:**<br> Изучив код источника, обнаружили, что есть дублирующиеся ссылки, как например "ТОМ ПЕРВЫЙ", "Часть первая", "I" имеют одну сслыку. Если дубли чистить методом set (множеством), то теряется порядок страниц, и их невозможно потом отсортировать, так как порядок идет по возрастанию левых симолов, таких как 1, 10, 100. Т.е. вторая страница будет только после всех сотых.<br>
*Решение:* Дополнить проверкой функции которая сравнивает с предидущим индексом.<br>

**Загвозка 2:**<br>
*Решение:*<br>
**Загвозка 3:**<br>
*Решение:*<br>
**Загвозка 4:**<br>
*Решение:*<br>
**Загвозка 5:**<br>
*Решение:*<br>
**Загвозка 6:**<br>
*Решение:*<br>
**Загвозка 7:**<br>
*Решение:*<br>
**Загвозка 8:**<br>
*Решение:*<br>
**Загвозка 9:**<br>
*Решение:*<br>

### Подключение библиотек и глобальных переменных

In [1]:
import requests # Бибилиотека запросов к web-сервисам
from bs4 import BeautifulSoup as bs # Библиотека парсинга web-страниц
import pandas as pd # Библиотека для работы с таблицами

# Для обработки текста
import nltk # Библиотека для работы с текстом
from nltk.tokenize import word_tokenize # , sent_tokenize токенайзер - разбивает непрерывный текст на токены(слова). word_tokenize - для разделения слов на токены. sent_tokenize - для разделения предложений на токены 
#from nltk.stem.snowball import SnowballStemmer # стеммер - преобразует тексты к начальной форме
#from nltk.corpus import stopwords # список незначимых слов - я, ты, он, она, там и тд !!! Много лишнего и занчимого
#nltk.download('all')
nltk.download('punkt')
#nltk.download('stopwords')

import string # используем для получения знаков пунктуации

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# Глобальные переменные
URL = 'https://ilibrary.ru' # + index.html - Страница оглавления книг "Война и Мир"
URL_CHAPTER  = '/text/11/index.html' # страница оглавления
URL_ENDINGS = [] # Список url страниц книги
TEXT_BOOK = '' # Обработанный текст для анализа слогов в виде списка

### Этап 1: Находжение источника

In [6]:
# Подключение к странице оглавления
connect_url = requests.get(URL + URL_CHAPTER)
if connect_url.status_code == 200:
  text_on_page = connect_url.text
else:
  print('СТРАНИЦА НЕ ДОСТУПНА')

In [7]:
# Функция проверка существующей ссылки
def check_href(href_url):
  ''' Проверка значение с пердидущим значением списка'''
  global URL_ENDINGS

  if len(URL_ENDINGS) < 1:
    URL_ENDINGS.append(href_url)
  else:
    if href_url != URL_ENDINGS[-1]:
      URL_ENDINGS.append(href_url)

In [8]:
# Получение всех страниц книги
chapter_page = bs(text_on_page, 'html.parser')
href_to_url = chapter_page.find_all('p')
for href_one in href_to_url:
  check_href(href_one.a['href'])

# Проверка количества страниц в списке, должно быть 361
print(len(URL_ENDINGS))

361


### Этап 2: Обработка сырых данных

In [9]:
# Списки пунктуации и стоп-слов (предлоги, местоимения и т.п.)
my_stop_punct = list(string.punctuation)
my_stop_punct = my_stop_punct + ['—'] + ['..'] + ['...'] + ['«'] + ['»']
my_stop_words = ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'ее', 'мне', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'ну', 'ли', 'уже', 'или', 'ни', 'него', 'до', 'вас', 'нибудь', 'уж', 'вам', 'там', 'ей', 'они', 'тут', 'где', 'есть', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'сам', 'чтоб', 'без', 'будто', 'чего', 'тоже', 'себе', 'под', 'ж', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'ним', 'этом', 'мой', 'тем', 'чтобы', 'нее', 'куда', 'зачем', 'всех', 'при', 'об', 'хоть', 'после', 'над', 'тот', 'эти', 'нас', 'про', 'них', 'какая', 'разве', 'эту', 'моя', 'свою', 'этой', 'чуть', 'том', 'такой', 'им', 'более', 'всю', 'это']

In [10]:
# Временное решение, чтоб не парсить 361 страницу сразу
URL_ENDINGS = URL_ENDINGS[:2]
print(len(URL_ENDINGS))

2


In [11]:
# Функция обработки текста
def text_processing(text, punkt, swort):
  ''' Принимает сырой текст, фильтрует его от знаков пунктуации, стоп-слов и переводит в нижний регистр'''
  
  words = word_tokenize(text.lower()) # Разбмваем на список слов (токены)
  words = [word for word in words if word not in punkt] # Удаляем знаки пунктуации
  words = [word for word in words if word not in swort] # Удаляем стопслова

  all_words = ' '.join(words)
  return all_words

In [12]:
%%time
# Получение текта книги с каждой страницы
for URL_ENDING in URL_ENDINGS:
  print(URL + URL_ENDING)
  connect_url_page = requests.get(URL + URL_ENDING)
  if connect_url_page.status_code == 200:
    text_page = connect_url_page.text

    one_page = bs(text_page, 'html.parser')
    text_in_url = one_page.find_all('span', class_='p')
    for text_one in text_in_url:
      TEXT_BOOK = TEXT_BOOK + text_processing(text_one.text, my_stop_punct, my_stop_words) + ' ' # Текст в функцию передается построчно до красной строки
      
  
  else:
    print('ALARM!!!STOP')


https://ilibrary.ru/text/11/p.1/index.html
https://ilibrary.ru/text/11/p.2/index.html
CPU times: user 186 ms, sys: 11.9 ms, total: 198 ms
Wall time: 2.78 s


####ТЕСТИРОВАНИЕ ВЫБОРКИ СЛОВ НАЧАТЬ ОТ СЮДА, ЧТОБ НЕ ГРУЗИТЬ ВСЕ ВРЕМЯ С САЙТА

In [30]:
#TEXT_BOOK_SRT = "лев толстой eh bien mon prince gênes et lucques ne sont plus que des apanages des поместья de la famille buonaparte non je vous préviens que si vous ne me dites pas que nous avons la guerre si vous vous permettez encore de pallier toutes les infamies toutes les atrocités de cet antichrist ma parole j y crois je ne vous connais plus vous n'êtes plus mon ami vous n'êtes plus верный раб comme vous dites 1. здравствуйте здравствуйте je vois que je vous fais peur 2 садитесь рассказывайте говорила июле 1805 года известная анна павловна шерер фрейлина приближенная императрицы марии феодоровны встречая важного чиновного князя василия первого приехавшего вечер анна павловна кашляла несколько дней был грипп говорила грипп был тогда новое слово употреблявшееся только редкими записочках разосланных утром красным лакеем было написано различия « si vous n'avez rien de mieux à faire monsieur le comte mon prince et si la perspective de passer la soirée chez une pauvre malade ne vous effraye pas trop je serai charmée de vous voir chez moi entre 7 et 10 heures annette scherer » 3 dieu quelle virulente sortie 4 отвечал нисколько смутясь такою встречей вошедший князь придворном шитом мундире чулках башмаках звездах светлым выражением плоского лица говорил изысканном французском языке котором только говорили думали наши деды теми тихими покровительственными интонациями которые свойственны состаревшемуся свете дворе значительному человеку подошел анне павловне поцеловал руку подставив надушенную сияющую лысину покойно уселся диване avant tout dites-moi comment vous allez chère amie 5 успокойте сказал изменяя голоса тоном котором из-за приличия участия просвечивало равнодушие даже насмешка можно быть здоровой когда нравственно страдаешь можно имея чувство оставаться спокойною наше время сказала анна павловна весь вечер надеюсь праздник английского посланника нынче середа надо показаться сказал князь дочь заедет мной повезет думала нынешний праздник отменен je vous avoue que toutes ces fêtes et tous ces feux d'artifice commencent à devenir insipides 6 ежели знали хотите праздник отменили сказал князь привычке заведенные часы говоря вещи которым хотел верили ne me tourmentez pas eh bien qu a-t-on décidé par rapport à la dépêche de novosilzoff vous savez tout 7 сказать сказал князь холодным скучающим тоном qu a-t-on décidé on a décidé que buonaparte a brûlé ses vaisseaux et je crois que nous sommes en train de brûler les nôtres 8 князь василий говорил всегда лениво актер говорит роль старой пиесы анна павловна шерер напротив несмотря свои сорок лет была преисполнена оживления порывов быть энтузиасткой сделалось общественным положением иногда когда даже хотелось обмануть ожиданий людей знавших делалась энтузиасткой сдержанная улыбка игравшая постоянно лице анны павловны хотя шла отжившим чертам выражала избалованных детей постоянное сознание своего милого недостатка которого хочет может находит нужным исправляться середине разговора политические действия анна павловна разгорячилась ах говорите австрию ничего понимаю может быть австрия никогда хотела хочет войны предает россия одна должна быть спасительницей европы наш благодетель знает свое высокое призвание будет верен вот одно верю нашему доброму чудному государю предстоит величайшая роль мире добродетелен хорош бог оставит исполнит свое призвание задавить гидру революции которая теперь ужаснее лице убийцы злодея одни должны искупить кровь праведника кого нам надеяться спрашиваю англия своим коммерческим духом поймет может понять высоту души императора александра отказалась очистить мальту хочет видеть ищет заднюю мысль наших действий сказали новосильцеву ничего поняли могли понять самоотвержения нашего императора который ничего хочет себя хочет блага мира обещали ничего обещали будет пруссия объявила бонапарте непобедим вся европа ничего может против верю одном слове гарденбергу гаугвицу cette fameuse neutralité prussienne ce n'est qu'un piège 9. верю одного бога высокую судьбу нашего милого императора спасет европу вдруг остановилась улыбкой насмешки своею горячностью думаю сказал князь улыбаясь ежели послали вместо нашего милого винценгероде взяли приступом согласие прусского короля красноречивы дадите чаю сейчас a propos прибавила опять успокоиваясь нынче два очень интересные человека le vicomte de mortemart il est allié aux montmorency par les rohans 10 одна лучших фамилий франции один хороших эмигрантов настоящих потом l'abbé morio 11 знаете глубокий ум был принят государем знаете очень рад буду сказал князь скажите прибавил только вспомнив что-то особенно-небрежно тогда спрашивал было главной целью посещения правда l'impératrice-mère 12 желает назначения барона функе первым секретарем вену c'est un pauvre sire ce baron à ce qu'il paraît 13 князь"
TEXT_BOOK_SRT = "одна лучших фамилий франции глубокий ум был принят государем знаете очень рад буду сказал князь скажите прибавил только вспомнив тогда спрашивал было главной"

In [44]:
len(TEXT_BOOK_SRT.split(' '))

23

###Этап 3: Внесение данных для нахождения слогов

In [88]:
vowels = 'ауоыиэяюёе' # гласные буквы
consonants = 'бвгджзклмнпрстфхцчшщ' # согласные буквы
TOTAL_SLOG = [] # будем создавать список с вложеным списком слогов, для подсчета совпадений и длин слов

In [89]:
# Получим возможные слоги
for consonant in consonants:
  for vowel in vowels:
    TOTAL_SLOG.append([consonant + vowel])

In [90]:
len(TOTAL_SLOG)

200

###Этап 4: Обработка полезных данных

In [93]:
def word_processing(index_word, word_word, len_word):
  ''' Функция поиска слогов в слове'''

  global vowels, consonants, TOTAL_SLOG

  for index_letter in range(1, len(word_word)):
    if word_word[index_letter] in vowels and word_word[index_letter - 1] in consonants:
      print('Слог: ' + word_word[index_letter - 1] + word_word[index_letter])

In [94]:
%%time
index_list = 0
for word_from_book in TEXT_BOOK_SRT.split(' '):
  word_processing(index_list, word_from_book, len(word_from_book))
  index_list += 1
  

Слог: на
Слог: лу
Слог: ши
Слог: фа
Слог: ми
Слог: ли
Слог: ра
Слог: ци
Слог: лу
Слог: бо
Слог: ки
Слог: бы
Слог: ри
Слог: ня
Слог: го
Слог: су
Слог: да
Слог: ре
Слог: на
Слог: те
Слог: че
Слог: ра
Слог: бу
Слог: ду
Слог: ка
Слог: за
Слог: ня
Слог: ка
Слог: жи
Слог: те
Слог: ри
Слог: ба
Слог: ви
Слог: то
Слог: ко
Слог: по
Слог: ни
Слог: то
Слог: да
Слог: ра
Слог: ши
Слог: ва
Слог: бы
Слог: ло
Слог: ла
Слог: но
CPU times: user 8.57 ms, sys: 0 ns, total: 8.57 ms
Wall time: 10.6 ms


***ЧЕРНОВИК***

In [1]:
a = 'Прив я тект.'

In [2]:
print(a)

Прив я тект.


In [4]:
a += a

In [5]:
print(a)

Прив я тект.Прив я тект.
